In [1]:
import os
import re
import glob 
import tables
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm
import matplotlib.pyplot as plt

In [2]:
TSUN = 4.9169e-6
gbf_cols = ['f','colat','lon','A','i','psi','phi','fdot','fddot','s_df/f','s_colat','s_lon','s_dA/A','s_i','s_psi','s_phi','s_dfdot/fdot','s_dfddot/fddot','s_Omega','SNR','b','l','Mc','d'] 
gbf = pd.read_table('../ldasoft/gbfisher_example/gbfisher_parameters.dat',delimiter=' ', header=None,index_col=False,names=gbf_cols)
gbf['i'] = 90 - abs(np.degrees(gbf['i']) - 90)
gbf['s_i'] = 90 - abs(np.degrees(gbf['s_i']) - 90)
gbf['s_Mc'] = (1/(5*TSUN))*((5/3)**(3/5))*(np.pi**(-8/5))*(gbf['f']**(-11/5))*(gbf['fdot']**(3/5))*((121*(gbf['s_df/f']**2) + 9*(gbf['s_dfdot/fdot']**2))**(1/2)) 
gbf.head()

In [3]:
def readData(raw, index, inFunc):
    x = []
    for pos, line in enumerate(raw):
        values = [float(x) for x in line.split()]
        x.append(inFunc(values[index]))
    x_50, x_10, x_90 = np.percentile(x,50), np.percentile(x,10), np.percentile(x,90)
    return [x_50, x_90-x_50, x_50-x_10]

def combineData(df, p):
    data = []; mx = []; ux = []; lx = []
    columns = ['B_n'] + [s for s in list(df.keys()) if re.match(p,s)]
    for j in range(df.shape[0]):
        mx.append(df[columns[1]][j])
        ux.append(df[columns[2]][j])
        lx.append(df[columns[3]][j])
        if(df['O_n'][j] == df['O_n'].max()):
            data.append([df['B_n'][j], np.percentile(mx,50), np.percentile(ux,50), np.percentile(lx,50)])
            mx = []; ux = []; lx = []
    return pd.DataFrame(data,columns=columns)

In [4]:
v = 'i'
exclude = ['fdot','inc']
prior_types = list(filter(lambda vp: re.search('(.*?)-gwprior', vp), os.listdir(os.getcwd())))
prior = [re.match('(.*?)-gwprior',s).group(1) for s in prior_types]
fix_incl = lambda i: 90 - abs(i - 90)

for n,p in enumerate(prior):
    binDir = os.path.join(os.getcwd(),f'{p}-gwprior')
    data = []
    for binary in os.listdir(binDir):
        for foldername in os.listdir(os.path.join(binDir,binary)):
            if os.path.isdir(os.path.join(binDir,os.path.join(binary,foldername))) & (not foldername in exclude):
                B_n = int(re.search('binary(.+?)row',foldername).group(1))
                O_n = int(re.search('row(.+?)_incl',foldername).group(1))
                file = os.path.join(binDir,os.path.join(binary,os.path.join(foldername,'post_equal_weights.dat')))
                with open(file,'r') as raw:
                    data.append([B_n, O_n]+readData(raw,4,fix_incl))

    gwl_cols = ['B_n', 'O_n', f'{p}_{v}', f'{p}_+s_{v}', f'{p}_-s_{v}']
    raw_gwl = pd.DataFrame(data,columns=gwl_cols)
    raw_gwl = raw_gwl.sort_values(['B_n','O_n']).reset_index(drop=True)
    sub_gwl = combineData(raw_gwl, p)
    if n == 0: gwl = sub_gwl; continue
    gwl = pd.merge(gwl, sub_gwl, on=['B_n'])

comb = pd.concat([gbf[[f'{v}',f's_{v}']], gwl], axis=1)
comb = comb.sort_values(['B_n']).reset_index(drop=True).drop(['B_n'], axis=1)
comb.head()

In [5]:
v = 'i'
nbins = 200
chain_cols = ['f','fdot','A','colat','lon','i','psi','phi']
chainDir = '../data/08yr_sp32_100_binaries/'
data = []

for binary in os.listdir(chainDir):
    B_n = int(re.search('binary(.*)',binary).group(1))
    chain = pd.read_table(os.path.join(chainDir,os.path.join(binary,'chains/dimension_chain.dat.1')),delimiter=' ',header=None,index_col=False,names=chain_cols)
    chain[f'{v}'] = 90 - abs(np.degrees(np.arccos(chain[f'{v}'])) - 90)
    mu = np.mean(chain[f'{v}'])
    std = np.std(chain[f'{v}'])
    n, bins = np.histogram(chain[f'{v}'], bins=nbins, density=True)
    kde = stats.gaussian_kde(chain[f'{v}'])
    data.append([B_n, n, bins, kde, mu, std])

gbm_cols = ['B_n',f'n_{v}',f'bins_{v}',f'kde_{v}',f'mu_{v}',f'std_{v}']
gbm = pd.DataFrame(data,columns=gbm_cols)
gbm = gbm.sort_values(['B_n']).reset_index(drop=True).drop(['B_n'], axis=1)
gbm.head()

In [6]:
v = 'i'
x_width = 4 #sigma
prior = ['old', 'normal', 'samples', 'kde']
colors = [f'C{t}' for t in range(0,len(prior))]
ub = 0; uc = 100

for j in range(ub,uc):
    plt.figure(j).set_size_inches(8.0,8.0)
    ax1 = plt.subplot(1, 2, 1)
    plt.title(f'Binary{str(j+1).zfill(3)} results',fontsize=20,pad=14)
    plt.ylabel('inclination (degrees)',fontsize=18)
    plt.xticks(np.arange(1,5),prior,rotation='vertical',fontsize=14)
    plt.yticks(fontsize=12)
    ax1.errorbar(x=range(0,6),y=comb[f'{v}'][j],linestyle='-',marker='',color='k',label='gbfisher data')
    plt.legend(fontsize=10)
    ax1.errorbar(x=range(0,6),y=comb[f'{v}'][j]+comb[f's_{v}'][j],linestyle='--',marker='',color='k')
    ax1.errorbar(x=range(0,6),y=comb[f'{v}'][j]-comb[f's_{v}'][j],linestyle='--',marker='',color='k')
    for n,p in enumerate(prior):
        ax1.errorbar(x=n+1,y=comb[f'{p}_{v}'][j],capsize=3,yerr=np.transpose(np.array(comb[[f'{p}_-s_{v}',f'{p}_+s_{v}']])[j:j+1]),linestyle='',marker='')   
    for xtick, color in zip(ax1.get_xticklabels(), colors):
        xtick.set_color(color)

    ax2 = plt.subplot(1, 2, 2)
    plt.title(f'Binary{str(j+1).zfill(3)} priors',fontsize=20,pad=14)
    plt.xlabel('inclination (degrees)', fontsize=18)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    plt.bar(gbm[f'bins_{v}'][j][:-1], gbm[f'n_{v}'][j], width=(gbm[f'bins_{v}'][j][-1]-gbm[f'bins_{v}'][j][0])/(len(gbm[f'bins_{v}'][j])-1),label='Chain Histogram')

    x = np.linspace(gbm[f'mu_{v}'][j]-x_width*gbm[f'std_{v}'][j],gbm[f'mu_{v}'][j]+x_width*gbm[f'std_{v}'][j],200)
    plt.plot(x,gbm[f'kde_{v}'][j](x),linewidth=3,label='KDE',color='C1')
    plt.plot(x,norm.pdf(x,gbm[f'mu_{v}'][j],gbm[f'std_{v}'][j]),linestyle='--',linewidth=3,label='Normal', color='C2')

    ymin, ymax = plt.ylim()
    plt.vlines(gbf[f'{v}'][j],ymin,ymax,label='True Inclination',linewidth=2,linestyle='-',color='k')
    plt.vlines(gbf[f'{v}'][j]+gbf[f's_{v}'][j],ymin,ymax,linewidth=2,linestyle='--',color='k')
    plt.vlines(gbf[f'{v}'][j]-gbf[f's_{v}'][j],ymin,ymax,linewidth=2,linestyle='--',color='k')

    plt.xlim(gbm[f'mu_{v}'][j]-x_width*gbm[f'std_{v}'][j], gbm[f'mu_{v}'][j]+x_width*gbm[f'std_{v}'][j])
    plt.legend(fontsize=10)

    plt.tight_layout(pad=4, rect=[0, 0.03, 1.7, 1])
    
    if not os.path.isdir('incl_plots'):
        os.makedirs('incl_plots')
    plotName = os.path.join('incl_plots',('binary'+str(j+1).zfill(3)+'_incl_plot.png'))
    plt.savefig(plotName, bbox_inches='tight')

In [7]:
# v = 'i'; lb = 0; ub = 100
# cut = [False]*lb + [True]*(ub-lb) + [False]*(100-ub)
# prior = ['old', 'normal', 'samples', 'kde']
# color = ['r','g','m','k']

# for n,p in enumerate(prior):
#     fig, ax = plt.subplots(figsize=(20,10))
#     ax.tick_params(axis='y', labelsize=14)
#     plt.xticks([])
#     plt.yticks(np.arange(0, 100, 10))
#     plt.ylabel('inclination (degrees)',fontsize=20)
#     plt.errorbar(x=range(lb,ub),y=comb[f'{v}'][cut],capsize=3,yerr=comb[f's_{v}'][cut],linestyle='',marker='.', color='C0')
#     plt.errorbar(x=np.arange(lb,ub,1)+0.05,y=comb[f'{p}_{v}'][cut],capsize=3,yerr=np.transpose(np.array(comb[[f'{p}_-s_{v}',f'{p}_+s_{v}']][cut])),linestyle='',marker='', color=color[n])   
#     legend = ['gbfisher data', f'gwemlisa {p} prior']
#     ax.legend(legend,fontsize=14)